<a href="https://colab.research.google.com/github/LuisEduardoRB/EDP-II/blob/main/Cantidad_econ%C3%B3mica_de_pedido_de_varios_art%C3%ADculos_con_limitaci%C3%B3n_de_almac%C3%A9n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explicación del bloque simbólico con Sympy

En este bloque se construye, de forma simbólica, el modelo de costo de inventario para **un solo artículo** y se deriva la expresión cerrada de la cantidad óptima de pedido en función del multiplicador de Lagrange.

---

## Importación de librerías

El código usa:

- `numpy` y `matplotlib` más adelante para cálculos y gráficas numéricas.
- `sympy` para hacer álgebra simbólica.
- `scipy.optimize.root_scalar` para resolver ecuaciones en una variable (no se usa todavía en este bloque, pero se usará después para encontrar el valor óptimo de $\lambda$).



In [41]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
from scipy.optimize import root_scalar

# ----- Símbolos para un artículo genérico -----
yi, Ki_s, Di_s, hi_s, ai_s, lam = sp.symbols('yi Ki Di hi ai lam', positive=True)

# Costo de un artículo: K_i D_i / y_i + h_i y_i / 2
TCU_i = Ki_s*Di_s/yi + hi_s*yi/2

# Condición de primer orden del lagrangiano: d/dy_i (TCU_i) - λ a_i = 0
eq = sp.Eq(sp.diff(TCU_i, yi) - lam*ai_s, 0)

# Resolver simbólicamente para y_i
sols_yi = sp.solve(eq, yi)

# Tomamos la raíz POSITIVA
sol_yi = sols_yi[1]   # [0] es la negativa, [1] la positiva

print("y_i(λ) simbólico:")
sp.pprint(sol_yi)

# Lambdify: función numérica y_i(K_i, D_i, h_i, a_i, λ)
y_lambda_scalar = sp.lambdify((Ki_s, Di_s, hi_s, ai_s, lam), sol_yi, 'numpy')


y_i(λ) simbólico:
                     _______________
     ____   ____    ╱      -1       
√2⋅╲╱ Di ⋅╲╱ Ki ⋅  ╱  ───────────── 
                 ╲╱   2⋅ai⋅lam - hi 


In [42]:
# ===== Datos de ejemplo =====
K = np.array([100.0, 120.0,  80.0])   # Costos de preparación K_i
D = np.array([500.0, 400.0, 600.0])   # Demandas D_i
h = np.array([  2.0,   2.5,  1.8])    # Costos de almacenamiento h_i
a = np.array([  0.5,   0.8,  0.6])    # Área por unidad a_i
A = 300.0                              # Capacidad total

n = len(K)


# Explicación completa del bloque numérico del modelo EOQ multiartículos con restricción de almacén

Este bloque de código implementa **la resolución numérica completa** del modelo EOQ multiartículos con limitación de capacidad de almacén, utilizando la expresión analítica obtenida previamente con *Sympy* y un método de búsqueda de raíces de *SciPy*. A continuación se explica de forma integral la **función de cada componente y la metodología global** que sigue el algoritmo.

---

## Modelo matemático de referencia

Se considera el problema de minimización del costo total anual de inventarios:

$ \min_{y_i>0}\; TCU(y)
= \sum_{i=1}^n\left(\frac{K_iD_i}{y_i} + \frac{h_i y_i}{2}\right) $

sujeto a la restricción de capacidad de almacén:

$ \sum_{i=1}^n a_i y_i \le A. $

---

## Función `TCU(y, K, D, h)`

Esta función calcula el **costo total anual de inventario** para un vector de cantidades de pedido $y = (y_1,\dots,y_n)$.

$ TCU(y) = \sum_{i=1}^n \left(\frac{K_iD_i}{y_i} + \frac{h_i y_i}{2}\right). $

En el código, la expresión se evalúa de manera vectorizada usando NumPy, lo que permite calcular eficientemente el costo total para todos los artículos simultáneamente.

---

## Función `y_lambda(lam_val, K, D, h, a)`

Esta función implementa la **fórmula cerrada** obtenida mediante el método de Lagrange para el caso en que la restricción de almacén está activa. Para cada artículo se tiene:

$ y_i(\lambda) = \sqrt{\frac{2K_iD_i}{h_i - 2\lambda a_i}}. $

La función recibe un valor candidato del multiplicador de Lagrange $\lambda$ y devuelve el vector completo $y(\lambda)$. Internamente utiliza la expresión simbólica convertida a función numérica mediante `lambdify`.

---

## Función `almacen_ecuacion(lam_val, K, D, h, a, A)`

Esta función representa la **restricción de capacidad** escrita como una ecuación escalar en términos de $\lambda$:

$ F(\lambda) = \sum_{i=1}^n a_i y_i(\lambda) - A. $

El objetivo es encontrar el valor $\lambda^*$ tal que $F(\lambda^*) = 0$, lo que implica que la capacidad del almacén se utiliza exactamente en el óptimo. Esta función es la que se pasa al método de búsqueda de raíces de SciPy.

---

## Función principal `resolver_multi_eoq_sympy_scipy(...)`

Esta función ejecuta **toda la metodología de solución** del modelo EOQ multiartículos con restricción de almacén:

1. **Conversión de datos**  
   Los parámetros $K, D, h, a$ se convierten a arreglos NumPy para permitir operaciones vectorizadas.

2. **Solución EOQ sin restricción**  
   Se calcula la solución clásica independiente para cada artículo:  
   $ y_i^{(0)} = \sqrt{\frac{2K_iD_i}{h_i}}. $  
   Luego se evalúa el uso de almacén asociado:  
   $ \sum_i a_i y_i^{(0)}. $

3. **Verificación de la restricción**  
   - Si $ \sum a_i y_i^{(0)} \le A $, la capacidad no es vinculante y la solución EOQ clásica es óptima.  
     En este caso se devuelve $y^{(0)}$, $\lambda^\*=0$ y el costo total mínimo.  
   - Si $ \sum a_i y_i^{(0)} > A $, la restricción es activa y se procede al método de Lagrange.

4. **Planteamiento del problema en $\lambda$**  
   Se define la función escalar  
   $ F(\lambda) = \sum a_i y_i(\lambda) - A. $  
   Teóricamente, $F(0) > 0$ y para $\lambda$ suficientemente negativo se tiene $F(\lambda) < 0$.

5. **Búsqueda de intervalo válido**  
   Se busca un intervalo $[\lambda_{\text{lo}}, 0]$ donde exista cambio de signo de $F(\lambda)$, condición necesaria para aplicar el método de bisección.

6. **Cálculo de $\lambda^*$ con SciPy**  
   Se utiliza `root_scalar` con el método de bisección para encontrar numéricamente $\lambda^*$ tal que:  
   $ \sum_{i=1}^n a_i y_i(\lambda^*) = A. $

7. **Obtención de la solución óptima**  
   Con $\lambda^*$ se calcula:  
   $ y_i^* = y_i(\lambda^*) $,  
   y posteriormente el costo mínimo $TCU(y^*)$.

La función devuelve:
- el vector óptimo de cantidades $y^*$,
- el multiplicador de Lagrange $\lambda^*$,
- el costo total mínimo,
- y un indicador lógico que señala si la restricción de almacén fue activa.



In [43]:
def TCU(y, K, D, h):
    return np.sum(K*D / y + 0.5*h*y)

def y_lambda(lam_val, K, D, h, a):
    # Usa la expresión simbólica positiva
    return y_lambda_scalar(K, D, h, a, lam_val)

def almacen_ecuacion(lam_val, K, D, h, a, A):
    y = y_lambda(lam_val, K, D, h, a)
    return np.dot(a, y) - A

def resolver_multi_eoq_sympy_scipy(K, D, h, a, A, tol=1e-6):
    K = np.array(K, float)
    D = np.array(D, float)
    h = np.array(h, float)
    a = np.array(a, float)

    # EOQ sin restricción
    y0 = np.sqrt(2.0*K*D / h)
    uso0 = np.dot(a, y0)

    if uso0 <= A + tol:
        # No se activa la restricción
        lam_opt = 0.0
        y_opt = y0
        tcu_opt = TCU(y_opt, K, D, h)
        return y_opt, lam_opt, tcu_opt, False

    # Restricción activa: buscar λ* < 0
    f = lambda lam_val: almacen_ecuacion(lam_val, K, D, h, a, A)

    # Seguridad: f(0) DEBE ser > 0 aquí
    f0 = f(0.0)
    if f0 <= 0:
        raise RuntimeError(f"Algo raro: f(0) = {f0}, no parece requerir restricción activa.")

    lam_hi = 0.0
    lam_lo = -1.0

    # Queremos f(lam_lo) < 0
    while f(lam_lo) > 0:
        lam_lo *= 2.0
        if lam_lo < -1e12:
            raise RuntimeError("No se pudo encontrar λ lo suficientemente negativo.")

    # Ahora sí: f(lam_lo) < 0 < f(0)
    sol = root_scalar(f, bracket=[lam_lo, lam_hi], method='bisect')
    lam_opt = sol.root

    y_opt = y_lambda(lam_opt, K, D, h, a)
    tcu_opt = TCU(y_opt, K, D, h)

    return y_opt, lam_opt, tcu_opt, True


In [44]:
y_opt, lam_opt, tcu_opt, restringido = resolver_multi_eoq_sympy_scipy(K, D, h, a, A)

print("¿Restricción de almacén activa?:", restringido)
print("λ* =", lam_opt)
print("\nCantidades óptimas y_i*:")
for i, yi in enumerate(y_opt, start=1):
    print(f"  Artículo {i}: y_{i}* = {yi:.4f}")

print("\nUso de almacén:")
print("  Σ a_i y_i* =", np.dot(a, y_opt))
print("  Capacidad A =", A)

print("\nTCU mínimo:", tcu_opt)


¿Restricción de almacén activa?: True
λ* = -1.3848501425363793

Cantidades óptimas y_i*:
  Artículo 1: y_1* = 171.8820
  Artículo 2: y_2* = 142.6789
  Artículo 3: y_3* = 166.5265

Uso de almacén:
  Σ a_i y_i* = 299.9999999999995
  Capacidad A = 300.0

TCU mínimo: 1415.6639089463097
